In [1]:
!pip install tensorflow keras scipy glob2

In [2]:
from keras.models import Model
from keras.layers import Flatten,Dense
from keras.applications.vgg16 import VGG16
import matplotlib.pyplot as plot
from glob import glob

In [3]:
IMAGESHAPE = [224, 224, 3]
training_data = 'chest_xray/train'
testing_data = 'chest_xray/test'

In [4]:
vgg_model = VGG16(input_shape=IMAGESHAPE, weights='imagenet', include_top=False)

58889256/58889256 [==============================] - 7s 0us/step


In [5]:
for each_layer in vgg_model.layers:
	each_layer.trainable = False

In [6]:
classes = glob('chest_xray/train/*')

In [7]:
flatten_layer = Flatten()(vgg_model.output)
prediction = Dense(len(classes), activation='softmax')(flatten_layer)

In [8]:
final_model = Model(inputs=vgg_model.input, outputs=prediction)
final_model.summary()


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      


                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0         
                                                                 
 block3_conv1 (Conv2D)       (None, 56, 56, 256)       295168    
                                                                 
 block3_conv2 (Conv2D)       (None, 56, 56, 256)       590080    
                                                                 
 block3_co

In [9]:
final_model.compile(
loss='categorical_crossentropy',
optimizer='adam',
metrics=['accuracy']
)


In [10]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
								shear_range = 0.2,
								zoom_range = 0.2,
								horizontal_flip = True)
testing_datagen = ImageDataGenerator(rescale =1. / 255)

In [11]:
import os
test_dir = "/content/chest_xray/test"

if os.path.exists(test_dir):
    print("Test folder found!")
    print("Contents:", os.listdir(test_dir))
else:
    print("Test folder not found. Extract the ZIP file again.")


Test folder not found. Extract the ZIP file again.


In [12]:
training_set = train_datagen.flow_from_directory('chest_xray/train',
												target_size = (224, 224),
												batch_size = 4,
												class_mode = 'categorical')


Found 5232 images belonging to 2 classes.


In [13]:
test_set = testing_datagen.flow_from_directory('chest_xray/test',
											target_size = (224, 224),
											batch_size = 4,
											class_mode = 'categorical')


Found 624 images belonging to 2 classes.


In [14]:
fitted_model = final_model.fit(
training_set,
validation_data=test_set,
epochs=5,
steps_per_epoch=len(training_set),
validation_steps=len(test_set)
)

Epoch 1/5
1308/1308 [==============================] - 2571s 2s/step - loss: 0.2746 - accuracy: 0.9226 - val_loss: 0.5492 - val_accuracy: 0.9087
Epoch 2/5
1308/1308 [==============================] - 2711s 2s/step - loss: 0.2384 - accuracy: 0.9440 - val_loss: 1.3427 - val_accuracy: 0.8077
Epoch 3/5
1308/1308 [==============================] - 3039s 2s/step - loss: 0.2340 - accuracy: 0.9524 - val_loss: 1.7898 - val_accuracy: 0.8109
Epoch 4/5
1308/1308 [==============================] - 2114s 2s/step - loss: 0.1831 - accuracy: 0.9623 - val_loss: 0.7072 - val_accuracy: 0.9119
Epoch 5/5
1308/1308 [==============================] - 2228s 2s/step - loss: 0.2213 - accuracy: 0.9589 - val_loss: 1.8600 - val_accuracy: 0.8077


In [15]:
final_model.save('our_model.h5')

c:\Users\sanja\AppData\Local\Programs\Python\Python38\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
from keras.preprocessing import image
from keras.models import load_model
from keras.applications.vgg16 import preprocess_input
import numpy as np
model=load_model('our_model.h5') #Loading our model
img = image.load_img('D:\\pd project\\Pneumonia Detection Dataset\\chest_xray\\train\\NORMAL\\NORMAL2-IM-1400-0001.jpeg',target_size=(224,224))
imagee=image.img_to_array(img) #Converting the X-Ray into pixels
imagee=np.expand_dims(imagee, axis=0)
img_data=preprocess_input(imagee)
prediction=model.predict(img_data)
if prediction[0][0]>prediction[0][1]: #Printing the prediction of model.
	print('Person is safe.')
else:
	print('Person is affected with Pneumonia.')
print(f'Predictions: {prediction}')

FileNotFoundError: [Errno 2] No such file or directory: 'D:\\pd project\\Pneumonia Detection Dataset\\chest_xray\\test\\NORMAL\\IM-1371-0001.jpeg'